In [1]:
import random
import numpy as np

from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Selectors
from pyevolve import Statistics

from geopy.distance import vincenty

In [2]:
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Fot collecting statistics
stats = []

coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))
    
matrix = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrix[i, j] = dist

In [3]:
def fitness(chromosome):
    score = 0
    for i in range(len(chromosome)):
        j = (i + 1) % len(chromosome)
        score += matrix[chromosome[i], chromosome[j]]
    return score

In [4]:
def current_best(ga_engine):
    stats.append(ga.getStatistics().asTuple())
    return False

In [5]:
def list_initializator(genome, **args):
    cities = [i for i in range(len(coords))]
    random.shuffle(cities)
    genome.setInternalList(cities)

In [6]:
def save_stats(filename):
    header = 'rawMax;rawMin;rawAve;rawDev;rawVar;fitMax;fitMin;fitAve'
    np.savetxt(filename, stats, '%.2f', delimiter=';', header=header, comments='')

In [12]:
# Chromosome representation

# genome = bitstring
genome = G1DList.G1DList(len(coords))
genome.initializator.set(list_initializator)

# how to compute the fitness
genome.evaluator.set(fitness)

genome.crossover.set(Crossovers.G1DListCrossoverEdge)

# GA initialisation
ga = GSimpleGA.GSimpleGA(genome)
ga.setPopulationSize(100)
ga.setMutationRate(0.01)
ga.setCrossoverRate(0.9)
ga.setElitism(True)
ga.setMinimax(Consts.minimaxType["minimize"])

# Number of generations
ga.setGenerations(300)

# In case we want to monitor the evolution process
# execute the function current_best every generation
ga.stepCallback.set(current_best)

ga.evolve(freq_stats=50)

# Final best solution
print(ga.bestIndividual())

save_stats('stats/edge_crossover.csv')

# [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8013.80(8301.26)/4782.55(4374.56)/6678.17(6678.17)]
Gen. 50 (16.67%): Max/Min/Avg Fitness(Raw) [5115.11(6055.07)/3827.01(3346.76)/4262.59(4262.59)]
Gen. 100 (33.33%): Max/Min/Avg Fitness(Raw) [5018.98(6099.94)/3817.90(3346.76)/4182.48(4182.48)]
Gen. 150 (50.00%): Max/Min/Avg Fitness(Raw) [5148.75(6051.56)/3830.67(3346.76)/4290.63(4290.63)]
Gen. 200 (66.67%): Max/Min/Avg Fitness(Raw) [5244.68(6313.54)/3909.97(3346.76)/4370.57(4370.57)]
Gen. 250 (83.33%): Max/Min/Avg Fitness(Raw) [5079.00(6871.06)/3948.34(3346.76)/4232.50(4232.50)]
Gen. 300 (100.00%): Max/Min/Avg Fitness(Raw) [5154.29(6523.68)/3929.61(3346.76)/4295.24(4295.24)]
Total time elapsed: 2.281 seconds.
- GenomeBase
	Score:			 3346.761974
	Fitness:		 3929.608979

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: fitness - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: list_initializator - Weight: 0.50
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  T